In [283]:
import torch
import torch.utils.data
import pandas as pd
import numpy as np
import src.starter.utils as starter
import src.starter.visualize as vis
import os
import h5py
import tqdm
from src.unet import UNet3D

In [286]:
from src.config import config

In [284]:
stnet = UNet3D(1, 3, False)

In [163]:
from sklearn.metrics import confusion_matrix

In [197]:
import src.utils as utils

In [198]:
def positions(im_shape, crop_shape, pads):
        zi, xi, yi = im_shape
        zc, xc, yc = crop_shape
        zp, xp, yp = pads
        for z in [z for z in range(0, zi, zp) if z + zc <= zi]:
            for x in [x for x in range(0, xi, xp) if x + xc <= xi]:
                for y in [y for y in range(0, yi, yp) if y + yc <= yi]:
                    yield (z, x, y)

In [203]:
pos = list(positions((128, 128, 128), (64, 64, 64), (32, 32, 32)))

In [208]:
arr = np.zeros((128, 128, 128))

In [212]:
z, x, y = pos[0]

In [217]:
predictions = np.random.randn(64, 64, 64)

In [279]:
stats = pd.read_csv("data_stats.csv")

In [291]:
stats.head()

,captured_pixel_width,captured_slice_thickness,case_id,case_nid,num_slices,height,width,im_volume,kidney_volume,tumor_volume,max_val,min_val
0,0.919922,1.0,case_00000,0,611.0,512.0,512.0,160169984.0,901262.0,18231.0,1413.0,-1024.0
1,0.798828,1.0,case_00001,1,602.0,512.0,512.0,157810688.0,1424126.0,21965.0,1393.0,-1024.0
2,0.939453,2.0,case_00002,2,261.0,512.0,512.0,68419584.0,606988.0,39224.0,3071.0,-1024.0
3,0.855469,2.0,case_00003,3,270.0,512.0,512.0,70778880.0,531769.0,13926.0,3071.0,-1024.0
4,0.976562,4.0,case_00004,4,64.0,512.0,512.0,16777216.0,113183.0,4932.0,2125.0,-1024.0


In [220]:
arr[z:z+64, x:x+64, y:y+64] = arr[z:z+64, x:x+64, y:y+64] + predictions

In [254]:
crops = pd.read_csv("crops.csv")

In [258]:
cases = crops.case_id.unique()

In [265]:
positions = [eval(pos) for pos in crops[crops.case_id == cases[0]].position.values]